In [ ]:
import os
import json
import PIL.Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from src.data.PascalVOC import dataset_voc, prepare_dataloaders_pascal_voc
from src.attacks.attacks import FastGradientSign, ProjectedGradientDescent, UniversalPerturbation
from src.training.Trainer import Trainer

In [ ]:
def load_model(num_classes = 20, model_path = None, to_cuda = True):
    if not model_path:
        model = torchvision.models.resnet18(pretrained = True)
        input_feat = model.fc.in_features
        
        model.fc = nn.Linear(input_feat, num_classes)
        model.fc.reset_parameters()
        loaded_state_dict = False
    
    else:
        print("Loaded", model_path)
        model = torchvision.models.resnet18()
        input_feat = model.fc.in_features
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_model = torch.load(model_path)
        model.load_state_dict(loaded_model['model_state_dict'])
        loaded_state_dict = True
        
    if to_cuda:
        model = model.to('cuda')
        
    return model, loaded_state_dict

In [ ]:
resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet_fine_tuned.pt')

for name, param in resnet.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

In [ ]:
root_dir = 'Data/PascalVOC/VOCdevkit/VOC2012'
config = {
    'lr': 0.005,
    'batch_size': 64,
    'weight_decay': 0.01
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = 500)

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [ ]:
for samp in dataloaders['train']:
    break

In [ ]:
images = samp['image']
labels = samp['label']

In [ ]:
uni_pert = UniversalPerturbation(resnet, loss_fn)

In [ ]:
vector = uni_pert.compute_perturbation(images, labels)